In [ ]:
pip install keras-tuner

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras
from kerastuner.tuners import RandomSearch
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/Updated_FinalFilteredCombined.csv')

# Display the first 5 rows
print(df.head())

# Print the column names and their data types
print(df.info())

   index  AADT_VN  BEGIN_POIN  COUNTY_COD  END_POINT  IRI_VN  IS_IMPROVED  \
0      0    44290         5.3          97        5.4      44            0   
1      1    44750         5.3          97        5.4      48            0   
2      2    45760         5.3          97        5.4      80            0   
3      3    48670         5.3          97        5.4      43            1   
4      4    49080         5.3          97        5.4      39            1   

       ROUTE_ID  SPEED_LIMI  THROUGH_LA  YEAR_RECOR  curval  \
0  IN0000100000        60.0           4        2013     263   
1  IN0000100000        60.0           4        2014     268   
2  IN0000100000        60.0           4        2015     192   
3  IN0000100000        70.0           4        2016     176   
4  IN0000100000        70.0           4        2017     205   

                                      geometry_paths  tmiles  tons  value  
0  [[[-88.31770505199995, 30.49976574300007], [-8...      15   716    270  
1  [[[

In [ ]:
df.index = range(1, len(df) + 1)
df.reset_index(inplace=True, names=['new_index'])  # Add 'new_index' as a column

# Method 2: Creating a new column directly
df['new_index'] = range(1, len(df) + 1)

# If you want 'index' instead of 'new_index'
df.rename(columns={'new_index': 'index'}, inplace=True)


In [ ]:
print(df.columns)

Index(['index', 'index', 'AADT_VN', 'BEGIN_POIN', 'COUNTY_COD', 'END_POINT',
       'IRI_VN', 'IS_IMPROVED', 'ROUTE_ID', 'SPEED_LIMI', 'THROUGH_LA',
       'YEAR_RECOR', 'curval', 'geometry_paths', 'tmiles', 'tons', 'value'],
      dtype='object')


In [ ]:
df['index'] = df['index'] + 1

In [ ]:
train_months = df['index']

In [ ]:
cols = ['AADT_VN', 'BEGIN_POIN', 'COUNTY_COD', 'END_POINT', 'IS_IMPROVED', 'SPEED_LIMI', 'THROUGH_LA', 'YEAR_RECOR', 'curval', 'tmiles', 'tons', 'value']
target_col = 'IRI_VN'

In [ ]:
df_for_training = df[cols + [target_col]]

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [ ]:
trainX = []
trainY = []

In [ ]:
n_future = 1
n_past = 8

In [ ]:
for i in range(n_past, len(df_for_training_scaled) - n_future + 1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:len(cols)])  # Only features
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, len(cols)])

In [ ]:
trainX, trainY = np.array(trainX), np.array(trainY)

In [ ]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (7111, 8, 12).
trainY shape == (7111, 1).


In [ ]:
print(trainY)

[[-0.80518051]
 [-0.32214006]
 [-0.48315354]
 ...
 [ 5.58168772]
 [ 1.43111789]
 [ 1.39533711]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

In [ ]:
import kerastuner as kt
from tensorflow.keras.regularizers import l2, l1, l1_l2

def build_tcn_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    l2_reg = hp.Float('l2_reg', min_value=1e-5, max_value=1e-3, sampling='log')

    model.add(Conv1D(filters=128, kernel_size=2, padding='causal', activation='relu', dilation_rate=1,
                     kernel_regularizer=l1(l2_reg)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Conv1D(filters=96, kernel_size=2, padding='causal', activation='relu', dilation_rate=2,
                     kernel_regularizer=l1(l2_reg)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Conv1D(filters=96, kernel_size=3, padding='causal', activation='relu', dilation_rate=4,
                     kernel_regularizer=l1(l2_reg)))
    model.add(Dropout(hp.Float('dropout_3', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Conv1D(filters=96, kernel_size=3, padding='causal', activation='relu', dilation_rate=4,
                     kernel_regularizer=l1(l2_reg)))
    model.add(Dropout(hp.Float('dropout_4', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu', kernel_regularizer=l1(l2_reg)))
    model.add(Dropout(0.1))
    model.add(Dense(trainY.shape[1]))

    model.compile(optimizer=Adam(learning_rate=0.00030508), loss='mse')
    return model

tuner = kt.RandomSearch(
    build_tcn_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='tcn_tuning',
    project_name='tcn_reg_dropout_1'
)

tuner.search(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(patience=10)])

tuner.results_summary()
best_model = tuner.get_best_models(num_models=1)[0]

Trial 20 Complete [00h 09m 49s]
val_loss: 0.48091359436511993

Best val_loss So Far: 0.43463167548179626
Total elapsed time: 02h 50m 51s
Results summary
Results in tcn_tuning/tcn_reg_dropout_1
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 15 summary
Hyperparameters:
l2_reg: 1.423617105148514e-05
dropout_1: 0.0
dropout_2: 0.2
dropout_3: 0.1
dropout_4: 0.0
Score: 0.43463167548179626

Trial 10 summary
Hyperparameters:
l2_reg: 1.2170633740470043e-05
dropout_1: 0.30000000000000004
dropout_2: 0.4
dropout_3: 0.4
dropout_4: 0.1
Score: 0.440905898809433

Trial 09 summary
Hyperparameters:
l2_reg: 3.6937225014755196e-05
dropout_1: 0.4
dropout_2: 0.30000000000000004
dropout_3: 0.30000000000000004
dropout_4: 0.0
Score: 0.46908338367938995

Trial 19 summary
Hyperparameters:
l2_reg: 5.794155666227713e-05
dropout_1: 0.2
dropout_2: 0.30000000000000004
dropout_3: 0.30000000000000004
dropout_4: 0.4
Score: 0.48091359436511993

Trial 05 summary
Hyperparameters:
l2_reg: 3.3218045

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch

def build_tcn_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    # TCN Blocks (you can add more)
    for i in range(hp.Int('num_tcn_blocks', min_value=2, max_value=5)):
        model.add(Conv1D(
            filters=hp.Int(f'filters_{i}', min_value=32, max_value=128, step=32),
            kernel_size=hp.Int(f'kernel_size_{i}', min_value=2, max_value=5),
            padding='causal',
            activation='relu',
            dilation_rate=2**i  # Example dilation
        ))
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Flatten())
    model.add(Dense(hp.Int('units_dense', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dense(trainY.shape[1]))

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss='mse'
    )
    return model

tuner = RandomSearch(
    build_tcn_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='tcn_tuning',  # Directory to save tuning results
    project_name='my_tcn_tuning'
)

tuner.search_space_summary()

tuner.search(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=10)]
)

tuner.results_summary()

best_model = tuner.get_best_models(num_models=1)[0]

Reloading Tuner from tcn_tuning/my_tcn_tuning/tuner0.json
Search space summary
Default search space size: 18
num_tcn_blocks (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
kernel_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
kernel_size_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units_dense (Int)
{'default': None, '

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score

def build_tcn_model(input_shape, output_units):
    model = Sequential()
    model.add(Input(shape=input_shape))  # Input layer

    # Temporal Convolutional Blocks
    model.add(Conv1D(filters=32, kernel_size=3, padding='causal', activation='relu', dilation_rate=1))
    model.add(Conv1D(filters=32, kernel_size=3, padding='causal', activation='relu', dilation_rate=2))
    model.add(Conv1D(filters=32, kernel_size=3, padding='causal', activation='relu', dilation_rate=4))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))  # Dropout for regularization
    model.add(Dense(output_units))  # Output layer

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
    return model

# --- Usage in your code ---
input_shape = (X_train.shape[1], X_train.shape[2])  # (n_past, num_features)
output_units = trainY.shape[1]  # 1 (for your target variable)

tcn_model = build_tcn_model(input_shape, output_units)
tcn_model.summary()

# --- Callbacks for training ---
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# --- Train the TCN ---
tcn_history = tcn_model.fit(
    X_train, y_train,
    epochs=120,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    callbacks=[early_stopping]
)

# --- Evaluate ---
y_pred_tcn = tcn_model.predict(X_test)
mse_tcn = mean_squared_error(y_test, y_pred_tcn)  # Assuming y_test is still available
print(f"TCN Mean Squared Error: {mse_tcn}")

# --- R2 Score ---
y_train_pred_tcn = tcn_model.predict(X_train)
r2_train_tcn = r2_score(y_train, y_train_pred_tcn)
r2_test_tcn = r2_score(y_test, y_pred_tcn)

print(f'TCN Training R²: {r2_train_tcn}')
print(f'TCN Test R²: {r2_test_tcn}')

# --- Visualization ---  (Adapt as needed)
plt.plot(tcn_history.history['loss'], label='TCN Training Loss')
plt.plot(tcn_history.history['val_loss'], label='TCN Validation Loss')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.regularizers import l2, l1, l1_l2

model = Sequential()
model.add(LSTM(512, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, kernel_regularizer=l1(0.00001)))
model.add(LSTM(256, activation='relu', return_sequences=False, kernel_regularizer=l1(0.00001)))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1], kernel_regularizer=l1(0.00001)))

In [ ]:
learning_rate = 0.0004

optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')
model.summary()


In [ ]:
history = model.fit(X_train, y_train, epochs=200, batch_size=16, validation_split=0.1, verbose=1)

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()


In [ ]:
y_pred = model.predict(X_test)

y_pred_original = np.zeros((y_pred.shape[0], df_for_training.shape[1])) # Dummy array with correct number of columns
y_pred_original[:, -1] = y_pred[:, 0]  # Place predictions in the last column
y_pred_original = scaler.inverse_transform(y_pred_original)[:, -1]

In [ ]:
# Create a dummy array for y_test as well
y_test_original = np.zeros((y_test.shape[0], df_for_training.shape[1]))
y_test_original[:, -1] = y_test[:, 0]
y_test_original = scaler.inverse_transform(y_test_original)[:, -1]

In [ ]:
print(y_pred_original)
print(y_test_original)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label='Actual')
plt.plot(y_pred_original, label='Predicted')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Actual vs. Predicted')
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(y_test_original, y_pred_original)
print(f'Mean Squared Error: {mse}')

In [ ]:
from sklearn.metrics import r2_score

# Calculate R² for the training set
y_train_pred = model.predict(X_train)

y_train_pred_original = np.zeros((y_train_pred.shape[0], df_for_training.shape[1]))
y_train_pred_original[:, -1] = y_train_pred[:, 0]
y_train_pred_original = scaler.inverse_transform(y_train_pred_original)[:, -1]

y_train_original = np.zeros((y_train.shape[0], df_for_training.shape[1]))
y_train_original[:, -1] = y_train[:, 0]
y_train_original = scaler.inverse_transform(y_train_original)[:, -1]

r2_train = r2_score(y_train_original, y_train_pred_original)
print(f'R² score for training set: {r2_train}')

# Calculate R² for the test set
y_test_pred = model.predict(X_test)

y_test_pred_original = np.zeros((y_test_pred.shape[0], df_for_training.shape[1]))
y_test_pred_original[:, -1] = y_test_pred[:, 0]
y_test_pred_original = scaler.inverse_transform(y_test_pred_original)[:, -1]

y_test_original = np.zeros((y_test.shape[0], df_for_training.shape[1]))
y_test_original[:, -1] = y_test[:, 0]
y_test_original = scaler.inverse_transform(y_test_original)[:, -1]

r2_test = r2_score(y_test_original, y_test_pred_original)
print(f'R² score for test set: {r2_test}')